In [1]:
# Importar las librerías necesarias
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configurar la conexión a la base de datos
host = '<HOST>'
database = '<DATABASE>'
user = '<USER>'
password = '<PASSWORD>'
port = '<PORT>'

# Crear la cadena de conexión
connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'

# Crear la conexión
engine = create_engine(connection_string)

# Definir las tablas a cargar
tablas = ['dim_producto',  
          'fact_venta', 'fact_detalle_venta']

# Cargar las tablas en DataFrames
dataframes = {}
for tabla in tablas:
    query = f'SELECT * FROM {tabla}'
    dataframes[tabla] = pd.read_sql(query, engine)

In [3]:
# Preparación y limpieza de datos
# Seleccionar las columnas relevantes
productos = dataframes["dim_producto"][["producto_id", "precio_lista", "disponible_en_pos"]]
fac_ventas = dataframes["fact_venta"][["fecha_id", "venta_id", "monto_total", "empresa_id", "cliente_id"]]
detalle_ventas = dataframes["fact_detalle_venta"]

# Convertir 'fecha_id' a formato datetime
fac_ventas["fecha_id"] = pd.to_datetime(fac_ventas["fecha_id"], format="%Y%m%d")

# Filtrar ventas con monto total positivo y detalles con cantidad positiva
fac_ventas = fac_ventas[fac_ventas["monto_total"] > 0]
detalle_ventas = detalle_ventas[detalle_ventas["cantidad"] > 0]

# Obtener las últimas tres compras por cliente
fac_ventas = fac_ventas.sort_values(by=["cliente_id", "fecha_id"], ascending=[False, False])
fac_ventas = fac_ventas.groupby("cliente_id").head(3).reset_index(drop=True)

# Combinar ventas y detalles de ventas
ventas_totales = pd.merge(fac_ventas, detalle_ventas, on="venta_id", how="inner")

# Agrupar por cliente para obtener media de precios y productos comprados
ventas_totales_agrupado = ventas_totales.groupby("cliente_id").agg(
    media_precios=("precio_unitario", "mean"),
    prods_comprados=("producto_id", list)
).reset_index()

In [4]:
# Construcción del conjunto de datos para el modelo
# Crear una lista de todos los productos
todos_los_productos = productos["producto_id"].unique().tolist()

# Crear un DataFrame vacío para almacenar los datos del modelo
datos = []

# Iterar sobre cada cliente
for _, row in ventas_totales_agrupado.iterrows():
    cliente = row["cliente_id"]
    media = row["media_precios"]
    prods_comprados = row["prods_comprados"]
    
    for prod in todos_los_productos:
        # Verificar si el cliente ha comprado el producto
        lo_ha_comprado = 1 if prod in prods_comprados else 0
        
        # Obtener información del producto
        producto_info = productos[productos["producto_id"] == prod].iloc[0]
        precio = producto_info["precio_lista"]
        disponible_pos = producto_info["disponible_en_pos"]
        
        # Verificar si el precio es menor o igual a la media de compras del cliente
        precio_menor_igual = 1 if precio <= media else 0
        
        # Agregar la fila al conjunto de datos
        datos.append([
            cliente, prod, lo_ha_comprado, precio_menor_igual,
            precio, media, disponible_pos
        ])

# Crear el DataFrame final para el modelo
df_modelo = pd.DataFrame(datos, columns=[
    "cliente_id", "producto_id", "lo_ha_comprado",
    "precio_menor_igual", "precio", "media_compras", "disponible_pos"
])

In [5]:
# Modelo de Regresión Logística
# Definir las variables predictoras y la variable objetivo
X = df_modelo[['precio_menor_igual', 'precio', 'media_compras', 'disponible_pos']]
y = df_modelo['lo_ha_comprado']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Crear y entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy del modelo: {accuracy:.2f}')
print('Matriz de confusión:')
print(conf_matrix)

# Obtener reporte de clasificación
report = classification_report(y_test, y_pred)
print('Reporte de clasificación:')
print(report)

Accuracy del modelo: 0.51
Matriz de confusión:
[[10762 10193]
 [   80   118]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.99      0.51      0.68     20955
           1       0.01      0.60      0.02       198

    accuracy                           0.51     21153
   macro avg       0.50      0.55      0.35     21153
weighted avg       0.98      0.51      0.67     21153



In [6]:
# Obtener las probabilidades de predicción
y_prob = model.predict_proba(X_test)

# Mostrar las probabilidades junto con las predicciones y valores reales
prob_df = pd.DataFrame({
    'Prob_No_Comprado': y_prob[:, 0],
    'Prob_Comprado': y_prob[:, 1],
    'Predicción': y_pred,
    'Real': y_test.values
})

prob_df.head()

,Prob_No_Comprado,Prob_Comprado,Predicción,Real
0,0.704812,0.295188,0,0
1,0.429778,0.570222,1,0
2,0.443428,0.556572,1,0
3,0.436999,0.563001,1,0
4,0.549825,0.450175,0,0
